In [ ]:
#Dependencies

#pip install matplotlib
#pip install numpy
#pip install opencv-python

#Importing

import cv2
import numpy
import matplotlib.pyplot

In [ ]:
#More info at https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html#load-the-yolo-network

#CFG file at: https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
#Coco labels at: https://github.com/pjreddie/darknet/blob/master/data/coco.names
#Weights at: https://pjreddie.com/media/files/yolov3.weights

#Initializing Yolo3 network

yolo = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

#Importing labels

classes = []
with open("coco.names","r") as file:
    classes = [line.strip() for line in file.readlines()]

In [ ]:
#Read network layers
#Still got no idea what outputlayers specifically does

layer_names = yolo.getLayerNames()
outputlayers = [layer_names[i-1] for i in yolo.getUnconnectedOutLayers()]

#Setting colors for rectangles

colorBlue   = (0, 0, 255)
colorGreen = (0, 255, 0)
colorRed = (255, 0, 0)

In [ ]:
# loading the image

name = "Pictures/image1.jpg"
img = cv2.imread(name)
height, width, channels = img.shape
matplotlib.pyplot.imshow(img)

In [ ]:
#Converting image into BLOB

blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)
yolo.setInput(blob)
outputs = yolo.forward(outputlayers)

In [ ]:
class_ids = []
confidences = []
boxes = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = numpy.argmax(scores)
        confidence = scores[class_id]
        
        if (confidence > 0.5):
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
colors = numpy.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
for i in range(len(boxes)):
    if i in indices:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        start = (x,y)
        end = (x+w,y+h)
        cv2.rectangle(img, start, end, colorGreen, 4)
        cv2.putText(img, label, (x,y-20), cv2.FONT_HERSHEY_TRIPLEX, 4, colorRed, 4)

In [ ]:
matplotlib.pyplot.imshow(img)